#Setup

In [ ]:
!apt install build-essential libunbound-dev libidn2-dev libssl-dev cmake

!git clone https://github.com/getdnsapi/getdns.git

In [ ]:
!cd getdns && mkdir build

In [ ]:
!cd getdns && git submodule update --init

In [ ]:
!sudo apt-get install check

In [ ]:
!cd getdns/build && cmake .. && make && make install

In [ ]:
!apt update
!pip install getdns

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,858 kB]
Get:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,129 kB]
Get:13 https://ppa.launchpadcontent.net/graphi

#Validate DNS Resolver

In [ ]:
import getdns
import sys
import csv
import os

# Function to perform DNS resolution using getdns
def resolve_domain(domain, resolver_ip):
    try:
        # Create a context with the specified resolver
        context = getdns.Context()
        context.resolution_type = getdns.RESOLUTION_STUB
        context.upstream_recursive_servers = [
            {"address_data": resolver_ip, "address_type": "IPv4"}
        ]

        # Perform the DNS resolution
        results = context.address(name=domain)

        # Check if the query was successful
        if results.status == getdns.RESPSTATUS_GOOD:
            print(f"DNS resolution for {domain} succeeded using resolver {resolver_ip}.")
            for addr in results.just_address_answers:
                print(f"IP Address: {addr['address_data']}")
            return True
        else:
            print(f"DNS resolution for {domain} failed with status: {results.status} using resolver {resolver_ip}")
            return False
    except getdns.error as e:
        print(f"An error occurred: {e}", file=sys.stderr)
        return False

# Function to read resolver IPs from a CSV file without headers
def read_csv(file_path):
    resolver_ips = []
    try:
        with open(file_path, mode='r') as file:
            csv_reader = csv.reader(file)
            for row in csv_reader:
                resolver_ips.append(row[0])
    except FileNotFoundError:
        print(f"Error: The file {file_path} was not found.", file=sys.stderr)
    except Exception as e:
        print(f"An error occurred while reading the CSV file: {e}", file=sys.stderr)
    return resolver_ips

# Function to write successful resolver IPs to a CSV file
def write_csv(file_path, data):
    try:
        with open(file_path, mode='w', newline='') as file:
            csv_writer = csv.writer(file)
            for row in data:
                csv_writer.writerow([row])
    except Exception as e:
        print(f"An error occurred while writing to the CSV file: {e}", file=sys.stderr)

# Main function
def main():
    folder_path = 'zmap_results'  # Path to the folder containing the CSV files
    output_file_path = 'successful_resolvers.csv'
    domain = "google.com"  # Fixed domain to be resolved
    idx = 1

    successful_resolvers = []

    # Iterate over all files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            resolver_ips = read_csv(file_path)
            for resolver_ip in resolver_ips:
                if resolve_domain(domain, resolver_ip):
                    successful_resolvers.append((filename, resolver_ip))
        write_csv(f"{idx}_successful_resolvers.csv", successful_resolvers)
        idx = idx + 1
        successful_resolvers = []

if __name__ == "__main__":
    main()

DNS resolution for google.com failed with status: 902 using resolver 172.65.226.120
DNS resolution for google.com failed with status: 902 using resolver 34.111.148.144
DNS resolution for google.com failed with status: 902 using resolver 166.104.103.187
DNS resolution for google.com failed with status: 902 using resolver 38.207.211.102
DNS resolution for google.com failed with status: 902 using resolver 141.219.82.217
DNS resolution for google.com failed with status: 902 using resolver 103.118.203.159
DNS resolution for google.com failed with status: 902 using resolver 163.191.243.225
DNS resolution for google.com failed with status: 902 using resolver 131.118.76.99
DNS resolution for google.com failed with status: 902 using resolver 213.151.43.105
DNS resolution for google.com failed with status: 902 using resolver 152.30.245.62
DNS resolution for google.com succeeded using resolver 8.8.8.8.
IP Address: 2404:6800:4008:c05::8a
IP Address: 2404:6800:4008:c05::8b
IP Address: 2404:6800:400

#Get Geo Location

In [ ]:
!sudo apt-get install geoip-bin

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
geoip-bin is already the newest version (1.6.12-8).
0 upgraded, 0 newly installed, 0 to remove and 53 not upgraded.
GeoIP Country Edition: GB, United Kingdom


In [ ]:
!geoiplookup 125.211.207.212

GeoIP Country Edition: US, United States


In [ ]:
!curl ipinfo.io/125.211.207.212?token=04c76ddfcb630d

In [ ]:
!pip install ipinfo

In [ ]:
import ipinfo
import csv

# Initialize the IPinfo client with your API token
access_token = '04c76ddfcb630d'  # Replace with your actual IPinfo API token
handler = ipinfo.getHandler(access_token)

def get_location(ip_address):
    details = handler.getDetails(ip_address)
    return {
        'ip': ip_address,
        'hostname': ip_address,
        'city': details.city,
        'region': details.region,
        'country': details.country,
        'loc': details.loc,
        'asn': details.asn
    }

def read_ips_from_csv(file_path):
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        ips = [row[0] for row in reader]
    return ips

def write_locations_to_csv(locations, output_file):
    if not locations:
        print("No locations to write.")
        return

    keys = locations[0].keys()
    with open(output_file, 'w', newline='') as output:
        dict_writer = csv.DictWriter(output, fieldnames=keys)
        dict_writer.writeheader()
        dict_writer.writerows(locations)

def main(input_file, output_file):
    ips = read_ips_from_csv(input_file)
    locations = []

    for ip in ips:
        try:
            location = get_location(ip)
            locations.append(location)
            print(f"Location for {ip}: {location}")
        except Exception as e:
            print(f"Error getting location for {ip}: {e}")

    write_locations_to_csv(locations, output_file)
    print(f"Geolocation information saved to {output_file}")

if __name__ == "__main__":
    input_file = '3_successful_resolver.csv'  # Input CSV file containing resolver IPs, one per line
    output_file = '3_resolver_locations.csv'  # Output CSV file for locations
    main(input_file, output_file)

Location for 112.5.173.102: {'ip': '112.5.173.102', 'hostname': '112.5.173.102', 'city': 'Shenzhen', 'region': 'Guangdong', 'country': 'CN', 'loc': '22.5455,114.0683', 'asn': {'asn': 'AS9808', 'name': 'China Mobile Communications Group Co., Ltd.', 'domain': '10086.cn', 'route': '112.5.160.0/20', 'type': 'isp'}}
Location for 185.228.168.50: {'ip': '185.228.168.50', 'hostname': '185.228.168.50', 'city': 'Temecula', 'region': 'California', 'country': 'US', 'loc': '33.5217,-117.1286', 'asn': {'asn': 'AS205157', 'name': 'Daniel Cid', 'domain': 'danielselectric.com', 'route': '185.228.168.0/24', 'type': 'business'}}
Location for 124.127.202.206: {'ip': '124.127.202.206', 'hostname': '124.127.202.206', 'city': 'Beijing', 'region': 'Beijing', 'country': 'CN', 'loc': '39.9075,116.3972', 'asn': {'asn': 'AS4847', 'name': 'China Networks Inter-Exchange', 'domain': 'bta.net.cn', 'route': '124.127.192.0/18', 'type': 'isp'}}
Location for 103.45.122.39: {'ip': '103.45.122.39', 'hostname': '103.45.122.

#Check Certificate Validity

In [ ]:
!pip install pandas

In [ ]:
import socket
import ssl
import pandas as pd

# Function to check certificate validity for a given resolver IP
def check_certificate(ip, asn_info, count, domain):
    print(f"Checking certificate for {ip}...")
    try:
        # Create a socket and wrap it with SSL/TLS
        with socket.create_connection((ip, 853), timeout=5) as sock:
            with ssl.create_default_context().wrap_socket(sock, server_hostname=ip) as ssock:
                # Retrieve certificate and verify
                cert = ssock.getpeercert()
                if cert:
                    certificate_status = "Valid"
                    reason = ""
                else:
                    certificate_status = "Invalid"
                    reason = "No certificate found."

                # Store the result
                results.append({'IP': ip, 'ASN Info': asn_info, 'Count': count, 'Certificate Status': certificate_status, 'Reason': reason})
    except Exception as e:
        # Store the exception message
        certificate_status = "Invalid"
        reason = str(e)
        results.append({'IP': ip, 'ASN Info': asn_info, 'Count': count, 'Domain': domain, 'Certificate Status': certificate_status, 'Reason': reason})

# Read the CSV file containing IP data
df = pd.read_csv('3_ip_counts_per_asn.csv')

# List to store the results
results = []

# Iterate over each row in the DataFrame and check certificate for each resolver IP
for index, row in df.iterrows():
    ip = row['IP Address']
    asn_info = row['ASN Info']
    count = row['Count']
    domain = row['Domain']
    check_certificate(ip, asn_info, count, domain)

# Create a new DataFrame from the results
results_df = pd.DataFrame(results)

# Save the results to a new CSV file
results_df.to_csv('certificate_results.csv', index=False)

#Several Helper Functions for Data Organization

In [ ]:
import csv

def extract_ips_from_csv(file_path):
    ips = []
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            # Assuming each row contains a single string
            if row:
                entry = row[0]
                # Split the entry to extract the IP address
                ip = entry.split(",")[1].strip().strip("')")  # Extract the IP part
                ips.append(ip)
    return ips

def main():
    input_file_path = '3_successful_resolvers.csv'  # Replace with your CSV file path
    output_file_path = '3_successful_resolver.csv'

    # Extract IPs from the CSV file
    extracted_ips = extract_ips_from_csv(input_file_path)

    # Print the extracted IPs (for debugging purposes)
    print(extracted_ips)

    # Write the extracted IPs to a new CSV file
    with open(output_file_path, 'w', newline='') as file:
        writer = csv.writer(file)
        for ip in extracted_ips:
            writer.writerow([ip])

if __name__ == "__main__":
    main()

['112.5.173.102', '185.228.168.50', '124.127.202.206', '103.45.122.39', '211.99.12.33', '202.100.200.47', '103.164.105.134', '103.59.149.181', '120.132.17.191', '221.7.196.248', '115.155.178.40', '113.106.84.30', '125.43.68.101', '103.45.120.135', '59.110.247.5', '59.110.246.59', '218.65.178.248', '122.228.236.74', '183.236.15.115', '76.76.2.74', '103.45.116.154', '47.110.175.69', '202.121.102.70', '115.227.7.136', '39.96.129.117', '47.93.92.89', '120.195.36.125', '115.230.124.116', '220.85.141.1', '112.161.188.36', '219.153.12.112', '222.186.139.130', '115.236.100.61', '211.152.36.94', '118.212.187.174', '103.45.116.34', '124.158.155.59', '103.45.129.65', '79.138.39.202', '223.95.33.139', '58.49.150.98', '43.248.139.182', '123.99.192.79', '212.93.104.164', '18.168.226.200', '58.56.83.244', '101.37.40.168', '103.127.8.131', '120.204.233.118', '111.207.123.93', '115.238.252.23', '200.188.59.182', '180.201.101.69', '43.243.130.30', '39.107.197.88', '103.71.69.83', '112.6.101.108', '45.12

In [ ]:
import pandas as pd

# Load the CSV file
file_path = '2_resolver_locations.csv'
df = pd.read_csv(file_path)

# Group the data by country and save each group to a separate file
grouped = df.groupby('country')

# Save each group to a separate file
for country, group in grouped:
    output_file_path = f'/content/2_geo_country_based/{country}_resolver_locations.csv'
    group.to_csv(output_file_path, index=False)

print("Files have been saved.")


Files have been saved.


In [ ]:
import pandas as pd
from collections import Counter

# Read the CSV file containing IP data
df = pd.read_csv('3_resolver_locations.csv')

# Assuming the CSV has a column named 'Country' containing country names
# and a column named 'IP' containing IP addresses

# Count IP data based on country
country_counts = Counter(df['country'])

# Create a new DataFrame to store the counts
count_df = pd.DataFrame(list(country_counts.items()), columns=['Country', 'Count'])

# Save the count values for all countries to a new CSV file
count_df.to_csv('3_country_counts.csv', index=False)


In [ ]:
import pandas as pd
from collections import defaultdict

# Read the CSV file containing IP data
df = pd.read_csv('3_resolver_locations.csv')

# Assuming the CSV has a column named 'asn' containing ASN information
# and a column named 'IP' containing IP addresses

# Group IP addresses based on ASN
asn_groups = defaultdict(list)
domain_groups = defaultdict(list)
for index, row in df.iterrows():
    asn_info = eval(row['asn'])
    asn_key = asn_info['asn']
    asn_domain = asn_info['domain']
    asn_groups[asn_key].append(row['ip'])
    domain_groups[asn_key] = asn_domain

# Count IP addresses within each ASN group
asn_counts = {}
for asn, ips in asn_groups.items():
    asn_counts[asn] = len(ips)

# Create a list to store rows for the new DataFrame
rows = []
for asn, count in asn_counts.items():
    asn_info = asn
    for ip in asn_groups[asn]:
        rows.append({'IP Address': ip, 'ASN Info': asn_info, 'Count': count, 'Domain': domain_groups[asn]})

# Create a new DataFrame from the rows
count_df = pd.DataFrame(rows)

# Save the count values for each ASN group to a new CSV file
count_df.to_csv('3_ip_counts_per_asn.csv', index=False)


In [3]:
import pandas as pd

# Read the three CSV files
df1 = pd.read_csv('1_country_counts.csv')
df2 = pd.read_csv('2_country_counts.csv')
df3 = pd.read_csv('3_country_counts.csv')

# Concatenate the dataframes
df = pd.concat([df1, df2, df3])

# Group by 'Country' and sum the 'Count' column
result = df.groupby('Country', as_index=False)['Count'].sum()

# Sort by 'Count' column in descending order
result = result.sort_values(by='Count', ascending=False)

# Save the result to a new CSV file
result.to_csv('country_counts.csv', index=False)

print("Merged CSV file created successfully.")

Merged CSV file created successfully.


In [8]:
import pandas as pd

# Read the three CSV files
df1 = pd.read_csv('1_ip_counts_per_asn.csv')
df2 = pd.read_csv('2_ip_counts_per_asn.csv')
df3 = pd.read_csv('3_ip_counts_per_asn.csv')

# Select only the relevant columns
df1 = df1[['ASN Info', 'Domain', 'Count']]
df2 = df2[['ASN Info', 'Domain', 'Count']]
df3 = df3[['ASN Info', 'Domain', 'Count']]

# Deduplicate each DataFrame based on 'ASN Info', keeping only the first occurrence
df1_dedup = df1.drop_duplicates(subset='ASN Info', keep='first')
df2_dedup = df2.drop_duplicates(subset='ASN Info', keep='first')
df3_dedup = df3.drop_duplicates(subset='ASN Info', keep='first')

# Concatenate the deduplicated dataframes
df_combined = pd.concat([df1_dedup, df2_dedup, df3_dedup])

# Group by 'ASN Info' and 'Domain' and sum the 'Count' column
result = df_combined.groupby(['ASN Info', 'Domain'], as_index=False)['Count'].sum()

# Sort by 'Count' column in descending order
result = result.sort_values(by='Count', ascending=False)

# Save the result to a new CSV file
result.to_csv('merged_output.csv', index=False)

print("Merged and sorted CSV file created successfully.")


Merged and sorted CSV file created successfully.


In [9]:
import pandas as pd

# Step 1: Merge and Sort

# Read the three CSV files
df1 = pd.read_csv('1_certificate_results.csv')
df2 = pd.read_csv('2_certificate_results.csv')
df3 = pd.read_csv('3_certificate_results.csv')

# Concatenate the dataframes
df_combined = pd.concat([df1, df2, df3])

# Sort by 'ASN Info'
df_sorted = df_combined.sort_values(by='ASN Info')

# Save the sorted result to a new CSV file
df_sorted.to_csv('merged_sorted.csv', index=False)

print("Merged and sorted CSV file created successfully.")

# Step 2: Aggregate and Create New File

# Group by 'ASN Info', 'Domain', and 'Reason' and calculate the sum of 'Count' and 'Reason_Count'
df_aggregated = df_sorted.groupby(['ASN Info', 'Domain', 'Reason']).agg({
    'Count': 'sum',
    'Reason': 'count'
}).rename(columns={'Reason': 'Reason_Count'}).reset_index()

# Save the aggregated result to a new CSV file
df_aggregated.to_csv('aggregated_output.csv', index=False)

print("Aggregated CSV file created successfully.")


Merged and sorted CSV file created successfully.
Aggregated CSV file created successfully.
